In [43]:
#!/usr/bin/env python
import pandas as pd
import json
from datetime import datetime
from datetime import date
try:
    # For Python 3.0 and later
    from urllib.request import urlopen
except ImportError:
    # Fall back to Python 2's urllib2
    from urllib2 import urlopen

import requests
dict = {"google" : "googl", "target" : "tgt" , "walmart" : "wmt", "dell" : "dell", "amazon" : "amzn", 'tesla' : "tsla",
         "boeing" : "ba", "microsoft" : "msft", "ford" : "f", "apple":"aapl"}


In [44]:
def getNYT(stock):
    your_key = 'XrEpgzLniLeuVv9Rwai6PZfol6OhEN91'

    url = 'https://api.nytimes.com/svc/news/v3/content/all/all/title.json?api-key=' +your_key
    r = requests.get(url)
    json_data = r.json()
    jdata = json_data['results']
    daily_Headlines = []

    for x in jdata:
        if stock in x['title'].lower() :
            daily_Headlines.append(x['title'].lower())
    df1 = pd.DataFrame(daily_Headlines, columns = ["Ticker", "Date", "Headline"])
    return daily_Headlines

In [45]:
def f_modeling(stock):
    key = "f0448bd30a7028e245052fcf3caa0837"
    stocks = ['AAPL','AMZN','MSFT','TSLA','TGT','WMT','DELL','F','BA']
    f_output = []
    
    url = ("https://financialmodelingprep.com/api/v3/stock_news?tickers="+ dict[stock].upper() + "&apikey=" + key)
    data = get_jsonparsed_data(url)

    output = parse_data(data)
    df = pd.DataFrame(output, columns = ["Ticker", "Date", "Headline"])
    today = date.today()
    i = 0;
    for x in df['Date']:
        if x == today:
             if stock in df['Headline'][i].lower() or dict[stock] in df['Headline'][i].lower() or (stock == "google" and "alphabet" in df['Headline'][i].lower()):
                f_output.append( df['Headline'][i].lower())                
        i = i +1

    return f_output;

In [46]:
def get_jsonparsed_data(url):
    """
    Receive the content of ``url``, parse it as JSON and return the object.

    Parameters
    ----------
    url : str

    Returns
    -------
    dict
    """
    response = urlopen(url)
    data = response.read().decode("utf-8")
    return json.loads(data)


In [47]:

def parse_data(data):
    output = []
    for d in data:
        date = datetime.strptime(d['publishedDate'], '%Y-%m-%d  %H:%M:%S').date()
        output.append((d['symbol'],date,d['title']))
    return output


In [48]:
def getPrice(stock):
    key = "f0448bd30a7028e245052fcf3caa0837"
    stocks = ['AAPL','AMZN','MSFT','TSLA','TGT','WMT','DELL','F','BA']
    f_output = []
    
    url = ("https://financialmodelingprep.com/api/v3/historical-price-full/"+ dict[stock].upper() + "?apikey=" + key)
    data = get_jsonparsed_data(url)
    data = data['historical']
    open = data[0]['open']
    close = data[0]['close']
    change = close - open
    return change
    

In [49]:
def main():
    for x in dict:
        head1 = getNYT(x)
        head2 = f_modeling(x)
        head2.extend(head1)
        allH = []
        col = []
        last = []
        i = 0
        col.append("Label")
        col.append("Date")
        if getPrice(x) > 0:
            allH.append(1)
        else:
            allH.append(0)
        allH.append( date.today())
        for y in head2:
            allH.append(str(y).lower())
            col.append("top" + str(i + 1))
            i = i + 1

        last.append(allH)
        dfH = pd.DataFrame(last, columns = col)
        print(x)
        print(dfH)
    

In [50]:
main()

google
   Label        Date
0      0  2021-05-01
target
   Label        Date
0      1  2021-05-01
walmart
   Label        Date                                      top1
0      1  2021-05-01  this is walmart's second biggest country
dell
   Label        Date
0      0  2021-05-01
amazon
   Label        Date                                               top1  \
0      0  2021-05-01  6 reasons andy jassy could drive amazon's mark...   

                                              top2  \
0  why amazon is still planning for massive growth   

                                                top3  
0  these 24 words from amazon's cfo may mean bill...  
tesla
   Label        Date                      top1
0      1  2021-05-01  tesla's big hit and miss
boeing
   Label        Date                                         top1
0      1  2021-05-01  boeing stumbles again with weak q1 earnings
microsoft
   Label        Date                                               top1
0      1  2021-05-01  h